In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Basic liberaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


#Sklearn Packages
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error



#get the file paths
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
#Define a function to check the RMSE       
def check_RMSE (y_train ,train_prediction , y_test ,  test_predicition):
    print ('Root Mean squared error for the train data  =  ' , 
           mean_squared_error(y_train ,train_prediction , squared=False ))
    print ('Root Mean squared error for the test data  =  ' , 
           mean_squared_error(y_test ,test_predicition , squared=False ))

In [ ]:
df_test  = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
df_test.head(2)

In [ ]:
df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_items.head(2)

In [ ]:
df_train  = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_train.head(2)

In [ ]:
# add month number 34 to the test dataset
df_test['date_block_num'] = 34
df_test = df_test[['date_block_num' , 'shop_id' , 'item_id' ]]
df_test.head(2)

In [ ]:
# map the latest price for the items in the train data set to the test data set
item_price = dict(df_train.groupby('item_id')['item_price'].last().reset_index().values)
df_test['item_price'] = df_test.item_id.map(item_price)
df_test.head(2)

In [ ]:
df_train = df_train[df_train.item_id.isin (df_test.item_id)]
df_train = df_train[df_train.shop_id.isin (df_test.shop_id)]

In [ ]:
df_train = df_train.groupby(['date_block_num' , 'shop_id' , 'item_id']).agg({'item_price': 'last', 'item_cnt_day': 'sum'}).reset_index()
df_train.head(2)

In [ ]:
df_train['shop*item'] = df_train.shop_id *df_train.item_id
df_train.head(2)

In [ ]:
df_test['shop*item'] = df_test.shop_id *df_test.item_id
df_test.head(2)

In [ ]:
df_items.drop('item_name' , axis  = 1 , inplace = True)
item_cat = dict(df_items.values)

df_train['item_cat'] = df_train.item_id.map(item_cat)

df_train.head(2)

In [ ]:
#map the categories
df_test['item_cat'] = df_test.item_id.map(item_cat)
df_test.head(2)

In [ ]:
df = pd.concat([df_train , df_test])
#Normalize
df.item_price = np.log1p(df.item_price)
#fil l the missing
df.item_price = df.item_price.fillna(df.item_price.mean())
#rremove the outlier
df.item_cnt_day = df.item_cnt_day.apply(lambda x : 10 if x>10 else x)

In [ ]:
df_train = df[df.item_cnt_day.notnull()]
df_train.head(2)

In [ ]:
df_test = df[df.item_cnt_day.isnull()]
df_test.drop ('item_cnt_day' , axis = 1 , inplace  = True)
df_test.head(2)

In [ ]:
X = df_train.drop('item_cnt_day' , axis = 1).values
y = df_train.item_cnt_day.values

In [ ]:
SC = MinMaxScaler()
SC.fit(X)
X = SC.transform(X)

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(X , y , test_size = 0.30)

In [ ]:
reg = RandomForestRegressor(n_estimators=25 )
reg.fit(x_train,y_train)
train_prediction  = reg.predict(x_train)
test_predicition  = reg.predict(x_test)

check_RMSE (y_train ,train_prediction , y_test ,  test_predicition)

In [ ]:
knn = KNeighborsRegressor()
knn.fit(x_train,y_train)
train_prediction  = knn.predict(x_train)
test_predicition  = knn.predict(x_test)

check_RMSE (y_train ,train_prediction , y_test ,  test_predicition)

In [ ]:
lr = LinearRegression()
lr.fit(x_train,y_train)
train_prediction  = lr.predict(x_train)
test_predicition  = lr.predict(x_test)

check_RMSE (y_train ,train_prediction , y_test ,  test_predicition)

In [ ]:
X_submission =df_test.values
X_submission = SC.transform(X_submission)

In [ ]:
predection  = knn.predict(X_submission)
sample_submission  = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
sample_submission.item_cnt_month = predection
sample_submission.head(2)

In [ ]:
sample_submission.to_csv('submission.csv' , index = False)